# CNN using VGG16



To set this up properly, the rotten images must be put into a folder titled `rotten` and the fresh images titled `fresh`. To determine the ripeness of a fruit, we will fit another similar model to a separate training data.

##Imports

In [ ]:
import os 
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import layers 
from tensorflow.keras import Model 

## ML Portion

Access to my personal Google Drive
> Do add shortcut of "CS3244 Group 50" to your personal drive


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Dataset for freshness model 
base_dir = F'/content/gdrive/MyDrive/CS3244 Group 50'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

train_fresh_dir = os.path.join(train_dir, 'fresh')
train_rotten_dir = os.path.join(train_dir, 'rotten')
test_fresh_dir = os.path.join(test_dir, 'fresh')
test_rotten_dir = os.path.join(test_dir, 'rotten')

# Dataset for ripeness model 
ripeness_dir = os.path.join(base_dir, 'ripeness')
ripeness_train_dir = os.path.join(ripeness_dir, 'train')
ripeness_test_dir = os.path.join(ripeness_dir, 'test')

train_ripe_dir = os.path.join(ripeness_train_dir, 'ripe')
train_overripe_dir = os.path.join(ripeness_train_dir, 'overripe')
train_green_dir = os.path.join(ripeness_train_dir, 'underripe')
# Found training sets from google image lol
test_ripe_dir = os.path.join(ripeness_test_dir, 'ripe')
test_overripe_dir = os.path.join(ripeness_test_dir, 'overripe')
test_green_dir = os.path.join(ripeness_test_dir, 'underripe')

Verification For picture loading


Step 1: Image Augmentation

In [ ]:
# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale = 1.0/255.)

Step 2: Training and validation sets


In [ ]:
# Flow training images in batches of 20 using train_datagen generator
freshness_train_generator = train_datagen.flow_from_directory(train_dir, batch_size = 20, class_mode = 'binary', target_size = (224, 224))

# Flow validation images in batches of 20 using test_datagen generator
freshness_test_generator = test_datagen.flow_from_directory(test_dir,  batch_size = 20, class_mode = 'binary', target_size = (224, 224))

# Flow training images in batches of 20 using train_datagen generator
ripeness_train_generator = train_datagen.flow_from_directory(ripeness_train_dir, batch_size = 10, class_mode = 'categorical', target_size = (224, 224))

# Flow validation images in batches of 20 using test_datagen generator
ripeness_test_generator = test_datagen.flow_from_directory(ripeness_test_dir,  batch_size = 10, class_mode = 'categorical', target_size = (224, 224))

Found 10903 images belonging to 2 classes.
Found 2698 images belonging to 2 classes.
Found 255 images belonging to 3 classes.
Found 15 images belonging to 3 classes.


Step 3: Loading the Base Model

In [ ]:
# # I managed to achieve 98++% using MobileNetV2 instead of VGG16, using the code snippet below.

# from keras.applications import MobileNetV2

# base_model = MobileNetV2(input_shape = (224, 224, 3), # Shape of our images
# include_top = False, # Leave out the last fully connected layer
# weights = 'imagenet')

# for layer in base_model.layers:
#     layer.trainable = False

# x = layers.GlobalAveragePooling2D()(base_model.output)
# x = layers.Dense(512, activation='relu')(x)
# x = layers.Dense(512, activation='relu')(x)
# x = layers.Dense(1, activation='sigmoid')(x)

# freshnessModel = tf.keras.models.Model(base_model.input, x)

# freshnessModel.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.0001), loss = 'binary_crossentropy',metrics = ['acc'])

# freshnessModel.fit(freshness_train_generator, validation_data = freshness_test_generator, steps_per_epoch = 100, epochs = 10)

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

base_model = VGG16(input_shape = (224, 224, 3), # Shape of our images
include_top = False, # Leave out the last fully connected layer
weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False

58892288/58889256 [==============================] - 1s 0us/step


Step 4: Compile and Fit

In [ ]:
# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# # Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

freshnessModel = tf.keras.models.Model(base_model.input, x)

freshnessModel.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.0001), loss = 'binary_crossentropy', metrics = ['acc'])

Step 5: Fit model with training set

In [ ]:
freshnessModel.fit(freshness_train_generator, validation_data = freshness_test_generator, steps_per_epoch = 100, epochs = 10)

Epoch 1/10
100/100 [==============================] - 1212s 12s/step - loss: 0.6689 - acc: 0.7151 - val_loss: 0.3342 - val_acc: 0.8540
Epoch 2/10
100/100 [==============================] - 446s 4s/step - loss: 0.3902 - acc: 0.8335 - val_loss: 0.1900 - val_acc: 0.9248
Epoch 3/10
100/100 [==============================] - 378s 4s/step - loss: 0.3045 - acc: 0.8602 - val_loss: 0.1685 - val_acc: 0.9340
Epoch 4/10
100/100 [==============================] - 320s 3s/step - loss: 0.2846 - acc: 0.8874 - val_loss: 0.2526 - val_acc: 0.8977
Epoch 5/10
100/100 [==============================] - 267s 3s/step - loss: 0.3661 - acc: 0.8513 - val_loss: 0.2341 - val_acc: 0.8988
Epoch 6/10
100/100 [==============================] - 214s 2s/step - loss: 0.2806 - acc: 0.8933 - val_loss: 0.2423 - val_acc: 0.8944
Epoch 7/10
100/100 [==============================] - 183s 2s/step - loss: 0.2308 - acc: 0.9113 - val_loss: 0.1965 - val_acc: 0.9177
Epoch 8/10
100/100 [==============================] - 166s 2s/step 

In [ ]:
from keras.applications import MobileNetV2

base_model = MobileNetV2(input_shape = (224, 224, 3), # Shape of our images
include_top = False, # Leave out the last fully connected layer
weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False

9412608/9406464 [==============================] - 0s 0us/step


In [ ]:
x = layers.GlobalAveragePooling2D()(base_model.output)

# we add dense layers so that the model can learn more complex functions and classify for better results.
x = layers.Dense(512,activation='relu')(x) 

# dense layer 2
x = layers.Dense(512,activation='relu')(x) 

# final layer with sigmoid (can be replaced with softmax) activation
x=layers.Dense(3,activation='sigmoid')(x) 

ripenessModel = tf.keras.models.Model(base_model.input, x)

ripenessModel.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.0001), loss = 'categorical_crossentropy', metrics = ['acc'])

In [ ]:
ripenessModel.fit(ripeness_train_generator, validation_data = ripeness_test_generator, steps_per_epoch = 20, epochs = 10)

Epoch 1/10
20/20 [==============================] - 56s 3s/step - loss: 1.0495 - acc: 0.5206 - val_loss: 0.6137 - val_acc: 0.8000
Epoch 2/10
20/20 [==============================] - 14s 712ms/step - loss: 0.5219 - acc: 0.8141 - val_loss: 0.4212 - val_acc: 0.9333
Epoch 3/10
20/20 [==============================] - 7s 309ms/step - loss: 0.3329 - acc: 0.8925 - val_loss: 0.3569 - val_acc: 0.8667
Epoch 4/10
20/20 [==============================] - 6s 305ms/step - loss: 0.3006 - acc: 0.9240 - val_loss: 0.2029 - val_acc: 1.0000
Epoch 5/10
20/20 [==============================] - 6s 284ms/step - loss: 0.2187 - acc: 0.9319 - val_loss: 0.2306 - val_acc: 0.9333
Epoch 6/10
20/20 [==============================] - 6s 286ms/step - loss: 0.1705 - acc: 0.9218 - val_loss: 0.2220 - val_acc: 0.9333
Epoch 7/10
20/20 [==============================] - 6s 278ms/step - loss: 0.1251 - acc: 0.9650 - val_loss: 0.1200 - val_acc: 1.0000
Epoch 8/10
20/20 [==============================] - 6s 281ms/step - loss: 0.1

Step 6(a): Save `Freshness` Model

Commented out so we don't accidentally rewrite models

In [ ]:
# model_save_name = 'model_9596_with_dropping' # RENAME AS YOU LIKE
# path = F"/content/gdrive/My Drive/CS3244 Group 50/freshness model/{model_save_name}" 
# freshnessModel.save(path)

Step 6(b): Save `Ripeness` Model

In [ ]:
# model_save_name = 'model_1000' # RENAME AS YOU LIKE
# path = F"/content/gdrive/My Drive/CS3244 Group 50/ripeness model/{model_save_name}" 
# ripenessModel.save(path)

# CHECKPOINT FOR FUTURE EASE OF USE

## Imports and Load Models

In [ ]:
import os 
import tensorflow as tf 
import numpy as np
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import layers 
from tensorflow.keras import Model 
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
model_save_name = 'model_9652_with_dropping' # List of models available are under google drive
path = F"/content/gdrive/My Drive/CS3244 Group 50/freshness model/{model_save_name}"
freshnessPredictionModel = keras.models.load_model(path)

In [ ]:
model_save_name = 'model_1000_sigmoid_final'
path = F"/content/gdrive/My Drive/CS3244 Group 50/ripeness model/{model_save_name}"
ripenessPredictionModel = keras.models.load_model(path)

## Prediction

Note that images are placed in group google drive > predictions

In [ ]:
freshness = ["fresh", "rotten"]
ripeness = ["overripe", "ripe", "underripe"]
pred_dir = '/content/gdrive/MyDrive/CS3244 Group 50/predictions'

def prediction(filepath):
  image = tf.keras.preprocessing.image.load_img(os.path.join(pred_dir, filepath), target_size=(224,224))
  input_arr_freshness = np.array([keras.preprocessing.image.img_to_array(image)]) # I have no idea why rescaling by 1./255. will give wrong predictions
  input_arr_ripeness = np.array([keras.preprocessing.image.img_to_array(image)/255.])
  freshnessPrediction = freshnessPredictionModel.predict(input_arr_freshness)
  ripenessPrediction = ripenessPredictionModel.predict(input_arr_ripeness)
  arr = ripenessPrediction[0].tolist()
  print(filepath + ': ' + freshness[int(freshnessPrediction[0][0])] + ', ' + ripeness[arr.index(max(arr))])

In [ ]:
prediction('underriped-banana.jpg')

underriped-banana.jpg: fresh, underripe


In [ ]:
prediction('good-banana.jpg')

good-banana.jpg: fresh, ripe


In [ ]:
prediction('overripe-banana1.jpg')

overripe-banana1.jpg: rotten, overripe


In [ ]:
prediction('overripe-banana2.jpg')

overripe-banana2.jpg: rotten, overripe


In [ ]:
prediction('overripe-banana3.jpg')

overripe-banana3.jpg: rotten, overripe


In [ ]:
prediction('underripe-apple1.jpg')

underripe-apple1.jpg: fresh, underripe


In [ ]:
prediction('underripe-apple2.jpg')

underripe-apple2.jpg: fresh, underripe


In [ ]:
# RIPENESS WRONG 
prediction('good-apple.jpg')

good-apple.jpg: fresh, underripe


In [ ]:
prediction('overripe-apple1.jpg')

overripe-apple1.jpg: rotten, overripe


In [ ]:
# FRESHNESS AND RIPENESS WRONG
prediction('overripe-apple2.jpg')

overripe-apple2.jpg: fresh, ripe


In [ ]:
prediction('underripe-oranges1.jpg')

underripe-oranges1.jpg: fresh, underripe


In [ ]:
prediction('underripe-oranges2.jpg')

underripe-oranges2.jpg: fresh, underripe


In [ ]:
# RIPENESS WRONG
prediction('good-oranges.jpg')

good-oranges.jpg: fresh, overripe


In [ ]:
# FRESHNESS WRONG
prediction('overripe-oranges1.jpg')

overripe-oranges1.jpg: fresh, overripe


In [ ]:
# FRESHNESS WRONG
prediction('overripe-oranges2.jpg')

overripe-oranges2.jpg: fresh, overripe
